# How The Threat Landscape Changed in the Pandemic Year 

The purpose of this analysis is to analyze malicious domains submitted to url.abuse[.]ch for the year range of 2019 to 2020. The URL Haus project is an amazing resource used by security practicioners of all kinds to help share and provide insight into malicious urls discovered. The URLhaus project is created and managed by abuse[.]ch, a non profit security reasearcher that manages several additional projects to provide situational awareness and information to aid in internet security against malware.

**The goal of this analysis is to answer the following:**

- What type of malware is more prominant in the year of the pandemic vs the previous year
- Which tld is most seen this year related to malicious threats
- Based on analysis are there any specifc hunts organization can perform to identify traffic going to risky domains.
- Based on analysis are there any detections organizations can put in place to 

**Analysis Breakdown**
- Data Exploration 
- Data Cleaning
- Analysis 
- Reporting and Visualization

In [2]:
import pandas as pd
from csv import reader

#Data Exploration in prep for data cleaning. Converting items in list to quickly asses what the data looks like.


of = open("csv.txt")
rf = reader(of)
data = list(rf)

print(data[:10])


# Data preview of first 10 rows show some header data prior to columns. Will need to skip those rows prior to cleaning.


[['################################################################'], ['# abuse.ch URLhaus Database Dump (CSV)                         #'], ['# Last updated: 2020-12-29 15:44:07 (UTC)                      #'], ['#                                                              #'], ['# Terms Of Use: https://urlhaus.abuse.ch/api/                  #'], ['# For questions please contact urlhaus [at] abuse.ch           #'], ['################################################################'], ['#'], ['# id', 'dateadded', 'url', 'url_status', 'threat', 'tags', 'urlhaus_link', 'reporter'], ['944671', '2020-12-29 15:44:07', 'http://gbimkd.org/wp-includes/mzYzedwYUHNvfwTTy47Ey0o5tKUyazH0oXIs/', 'online', 'malware_download', 'doc,emotet,epoch2', 'https://urlhaus.abuse.ch/url/944671/', 'Cryptolaemus1']]


In [3]:
#Creating a dataframe , but will be skipping first 8 rows of header data.

urls = pd.read_csv("csv.txt",skiprows=8)

columns= urls.columns

print(urls.info(),"\n")



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 931755 entries, 0 to 931754
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   # id          931755 non-null  int64 
 1   dateadded     931755 non-null  object
 2   url           931755 non-null  object
 3   url_status    931755 non-null  object
 4   threat        931698 non-null  object
 5   tags          931755 non-null  object
 6   urlhaus_link  931755 non-null  object
 7   reporter      931755 non-null  object
dtypes: int64(1), object(7)
memory usage: 56.9+ MB
None 



## Data Exploration and Data Cleaning 

- Dataframe created successfully, identifying over 900,000 entries. 
- Datacleaning steps:
 1. Clean up columns to make them easier to work with
 2. Create a new column for year based on dateadded column
 3. Create a new column for month based on date added column
 4. Identify null values
 5. Create a new column for domain bsed or url
 

In [4]:
#loop through column headers to make naming convention more consistent

columns = urls.columns

print("old columns","\n", columns,"\n")

new_columns = []

for c in columns:
    new = c
    if c == "# id":
        new = "id"
        new_columns.append(new)
    elif c =="dateadded":
        new = "date_added"
        new_columns.append(new)
        
    else:
        new_columns.append(new)
        
urls.columns = new_columns


        
print("new columns", "\n",urls.columns,"\n")


print("data preview: \n",urls.head(2))


old columns 
 Index(['# id', 'dateadded', 'url', 'url_status', 'threat', 'tags',
       'urlhaus_link', 'reporter'],
      dtype='object') 

new columns 
 Index(['id', 'date_added', 'url', 'url_status', 'threat', 'tags',
       'urlhaus_link', 'reporter'],
      dtype='object') 

data preview: 
        id           date_added  \
0  944671  2020-12-29 15:44:07   
1  944670  2020-12-29 15:44:05   

                                                 url url_status  \
0  http://gbimkd.org/wp-includes/mzYzedwYUHNvfwTT...     online   
1                       http://116.124.219.2:55737/i     online   

             threat               tags                          urlhaus_link  \
0  malware_download  doc,emotet,epoch2  https://urlhaus.abuse.ch/url/944671/   
1  malware_download     32-bit,arm,elf  https://urlhaus.abuse.ch/url/944670/   

        reporter  
0  Cryptolaemus1  
1       geenensp  


### Parsing date_added column and creating seperate columns for year and month



In [5]:
import datetime 

urls["date_added"] = pd.to_datetime(urls["date_added"])

urls["year"] = urls["date_added"].dt.year
urls["month"] = urls["date_added"].dt.strftime("%b")

In [7]:
print(urls["month"].describe())

print(urls["year"].describe())


count     931755
unique        12
top          Sep
freq      202906
Name: month, dtype: object
count    931755.000000
mean       2019.608268
std           0.663945
min        2018.000000
25%        2019.000000
50%        2020.000000
75%        2020.000000
max        2020.000000
Name: year, dtype: float64


### Parsing url column to extract domain

- Data contained in URL column is a mixture of hardcoded IP's and fqdn's
- Will create a new column named domain_request:
 1. For rows where the URL is an IP address, I will indicate "IP" as domain_request
 2. Fore rows where the URL is a fqdn, I will extract only the tld and append in the column.

- I will use the following data cleaning workflow process
 1. Explore the data in the column
 2. Identify patterns and special cases
 3. Remove non applicable characters
 4. Perform any data type conversions necessary
 5. Rename column or create new column if required.
 
I've previewed the strings below and split them based on the forward slash delimeter as that is is a consisten pattern seen in all url request. i.e. http[s]://[hostname/ip]/[path]




In [29]:
print(urls["url"].str.split("/").head(30))

0     [http:, , gbimkd.org, wp-includes, mzYzedwYUHN...
1                     [http:, , 116.124.219.2:55737, i]
2     [http:, , www.ticketshd.com, wp-content, FUfYN...
3     [http:, , thefiercevagabond.com, cgi-bin, Ebob...
4     [http:, , 175.10.51.6:57578, Mozi.a;chmod+777+...
5     [https:, , www.hintup.com.br, wp-content, FE9Q...
6     [http:, , ondigital.one, wp-admin, up9pp9KLyef...
7     [https:, , hdgarden.vn, wp-includes, aZBWciJOr...
8                     [http:, , 115.56.159.28:57022, i]
9     [https:, , www.lixko.com, wp-includes, LEq9VJd, ]
10    [https:, , surfboarddigital.com, carol-stream-...
11             [https:, , srishtiherbs.com, jms, bq8, ]
12         [https:, , unikaryapools.com, wp, ysFiRq1, ]
13                  [http:, , 207.254.247.210:49250, i]
14                   [http:, , 120.57.103.125:55379, i]
15        [https:, , technicalashish.in, wp-admin, M, ]
16    [http:, , thespaceastronauts.com, wp-content, ...
17        [http:, , sanghuangvip.com, wp-admin, 

Based on the preview of data above, the hostname/ip is located at index 2.

- I will exstract the domain/ip and place in a newly created column named host

In [36]:
urls["host"]= urls["url"].str.split("/").str[2]

print(urls["host"].head(10))

0               gbimkd.org
1      116.124.219.2:55737
2        www.ticketshd.com
3    thefiercevagabond.com
4        175.10.51.6:57578
5        www.hintup.com.br
6            ondigital.one
7              hdgarden.vn
8      115.56.159.28:57022
9            www.lixko.com
Name: host, dtype: object


Some additional parsing will need to be completed to isolate the TLD.In the following section I will create an addition new column named tld. To accomplish this, I will need to do the following:
- Perform another split on the coloumn, but this split will be based on the "." dellimeter.
- I will append the last item in the index to the tld column
- Some challenges with the data and observations are
 1. some domains have two tld appended i.e. .org.com
 2. request to IP addresses also include port value.
 
- I will create a test dataframe with a subset of data. For fields that contain":", I will indicate that as an IP, for all others, I will simply take the last domain listed.


In [124]:
testdata= urls[["year","host"]].head(10)

#print(testdata)

testdata["tld"] = testdata["host"].str.split(".").str[-1]

#print(testdata)

filter = testdata["tld"].str.contains(":")



testdata.loc[filter,"tld"] = "ip"

print(testdata)






   year                   host  tld
0  2020             gbimkd.org  org
1  2020    116.124.219.2:55737   ip
2  2020      www.ticketshd.com  com
3  2020  thefiercevagabond.com  com
4  2020      175.10.51.6:57578   ip
5  2020      www.hintup.com.br   br
6  2020          ondigital.one  one
7  2020            hdgarden.vn   vn
8  2020    115.56.159.28:57022   ip
9  2020          www.lixko.com  com


Now that I've testing code on a subset of data, will apply to the full DB and previiew the data below

In [125]:
urls["tld"] = urls["host"].str.split(".").str[-1]

filter = urls["tld"].str.contains(":")

urls.loc[filter,"tld"] = "ip"

print(urls[["year","host","tld"]].head(20))

    year                    host  tld
0   2020              gbimkd.org  org
1   2020     116.124.219.2:55737   ip
2   2020       www.ticketshd.com  com
3   2020   thefiercevagabond.com  com
4   2020       175.10.51.6:57578   ip
5   2020       www.hintup.com.br   br
6   2020           ondigital.one  one
7   2020             hdgarden.vn   vn
8   2020     115.56.159.28:57022   ip
9   2020           www.lixko.com  com
10  2020    surfboarddigital.com  com
11  2020        srishtiherbs.com  com
12  2020       unikaryapools.com  com
13  2020   207.254.247.210:49250   ip
14  2020    120.57.103.125:55379   ip
15  2020      technicalashish.in   in
16  2020  thespaceastronauts.com  com
17  2020        sanghuangvip.com  com
18  2020     117.242.211.7:40745   ip
19  2020  nasabatam.bbtbatam.com  com


In [130]:
print(urls["tld"].describe())

count     931755
unique       843
top           ip
freq      501060
Name: tld, dtype: object
